In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from preprocessor import *
import torch
from copiedpreprocessor import *
from sklearn.model_selection import train_test_split
from models.mstcn import *
from models.resnet_feature_extractor import *
from FeatureExtractor import *

In [ ]:
ExtractFeaturesFromMouthFrames("processedDataset/realframes.npz", "featuresExtracted/realfeatures32.npz")
ExtractFeaturesFromMouthFrames("processedDataset/deepfakesframes.npz", "featuresExtracted/deepfakesfeatures32.npz")

In [2]:

ExtractFeaturesFromMouthFrames("processedDataset/face2faceframes.npz", "featuresExtracted/face2facefeatures.npz")
ExtractFeaturesFromMouthFrames("processedDataset/faceswapframes.npz", "featuresExtracted/faceswapfeatures.npz")
ExtractFeaturesFromMouthFrames("processedDataset/neuraltexturesframes.npz", "featuresExtracted/neuraltexturesfeatures.npz")

In [3]:
MSTCNModel = load_mstcn_model('pretrainedModels/mstcn.pth',configfile="models/configs/mstcn.json")

In [4]:
#Finetune model.tcn by training it on features extracted from real and fake videos
FakeFeaturesPath = 'featuresExtracted/deepfakesfeatures.npz'
RealFeaturesPath = 'featuresExtracted/realfeatures.npz'
fakeFeatures = np.load(FakeFeaturesPath)['features']
realFeatures = np.load(RealFeaturesPath)['features']
# fakeFeatures = fakeFeatures.reshape(fakeFeatures.shape[0], fakeFeatures.shape[2], fakeFeatures.shape[3])
# realFeatures = realFeatures.reshape(realFeatures.shape[0], realFeatures.shape[2], realFeatures.shape[3])
fakeLabels = np.zeros((fakeFeatures.shape[0],1))
realLabels = np.ones((realFeatures.shape[0],1))
features = np.concatenate((fakeFeatures, realFeatures), axis=0)
labels = np.concatenate((fakeLabels, realLabels), axis=0)
#shuffle features and labels
indices = np.arange(features.shape[0])
np.random.shuffle(indices)
features = features[indices]
labels = labels[indices]

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

trainloader = torch.utils.data.DataLoader(list(zip(X_train, y_train)), batch_size=32, shuffle=True)
testloader = torch.utils.data.DataLoader(list(zip(X_test, y_test)), batch_size=32, shuffle=True)

In [5]:
learning_rate = 2e-4
batch_size = 32

In [6]:
def calculateAccuracy(preds,labels):
   
    preds = np.array(preds).flatten()
    labels = np.array(labels).flatten()
    return (preds == labels).mean()

In [7]:
epochs = 10
optimizer = torch.optim.Adam(MSTCNModel.parameters(), lr=learning_rate)
criterion = nn.BCELoss()

for epoch in range(epochs):
    running_loss = 0.0
    all_labels = []
    all_preds = []
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs = inputs.cuda().float()
        labels = labels.cuda().float()
        optimizer.zero_grad()
        outputs = MSTCNModel(inputs, lengths=[inputs.shape[1] for i in range(inputs.shape[0])])
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        all_preds.extend(torch.round(outputs).detach().cpu().numpy())
        all_labels.extend(labels.detach().cpu().numpy())
        
    print('Epoch: %d, loss: %.3f' % (epoch + 1, running_loss / (i + 1)))
    print('accuracy: %.3f' % (calculateAccuracy(all_preds, all_labels)))

Epoch: 1, loss: 0.875
accuracy: 0.738
Epoch: 2, loss: 0.401
accuracy: 0.831
Epoch: 3, loss: 0.307
accuracy: 0.870
Epoch: 4, loss: 0.261
accuracy: 0.897
Epoch: 5, loss: 0.201
accuracy: 0.920
Epoch: 6, loss: 0.158
accuracy: 0.937
Epoch: 7, loss: 0.134
accuracy: 0.949
Epoch: 8, loss: 0.117
accuracy: 0.955
Epoch: 9, loss: 0.107
accuracy: 0.961
Epoch: 10, loss: 0.075
accuracy: 0.972


In [9]:
MSTCNModel.eval()

#Calculate accuracy on test set
all_labels = []
all_preds = []
for i, data in enumerate(testloader, 0):
    inputs, labels = data
    inputs = inputs.cuda().float()
    labels = labels.cuda().float()
    outputs = MSTCNModel(inputs, lengths=[inputs.shape[1] for i in range(inputs.shape[0])])
    all_preds.extend(torch.round(outputs).detach().cpu().numpy())
    all_labels.extend(labels.detach().cpu().numpy())
print('accuracy: %.3f' % (calculateAccuracy(all_preds, all_labels)))

accuracy: 0.898


In [10]:
torch.save(MSTCNModel.state_dict(), 'trainedclassifiers/mstcn_deepfakes_1.pth')